# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [22]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [23]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")

# drop rows containing any missing value
spaceship = spaceship.dropna()

# Cleaning the data
spaceship['Transported'] = spaceship['Transported'].astype(int)

# Column `Cabin` is too granular - transform it in order to obtain {'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'}
spaceship['Cabin'] = spaceship['Cabin'].str[0]

# Drop PassengerId and Name
spaceship = spaceship.drop(['PassengerId', 'Name'], axis=1)

# One-hot encoding for non-numerical columns
non_numerical_columns = spaceship.select_dtypes(include='object').columns

spaceship = pd.get_dummies(spaceship, columns=non_numerical_columns, drop_first=True)

spaceship.describe( include = 'all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Age,6606.0,NaN,NaN,NaN,28.894036,14.533429,0.0,19.0,27.0,38.0,79.0
RoomService,6606.0,NaN,NaN,NaN,222.991674,644.987936,0.0,0.0,0.0,49.0,9920.0
FoodCourt,6606.0,NaN,NaN,NaN,478.958523,1678.592291,0.0,0.0,0.0,82.75,29813.0
ShoppingMall,6606.0,NaN,NaN,NaN,178.356494,576.328407,0.0,0.0,0.0,30.0,12253.0
Spa,6606.0,NaN,NaN,NaN,313.16152,1144.016291,0.0,0.0,0.0,65.0,22408.0
VRDeck,6606.0,NaN,NaN,NaN,303.780048,1127.142166,0.0,0.0,0.0,52.0,20336.0
Transported,6606.0,NaN,NaN,NaN,0.503633,0.500025,0.0,0.0,1.0,1.0,1.0
HomePlanet_Europa,6606,2,False,4933,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HomePlanet_Mars,6606,2,False,5239,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CryoSleep_True,6606,2,False,4274,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [24]:
X = spaceship.drop('Transported', axis=1)
y = spaceship['Transported']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=0)

# 1. Normalizamos: KNN, SVM, Neural Networks (Modelos de distancia)
# 2. Standarization: Regressiones Lineales
# 3. No normalizar/ni estandarizar (Ensamblados): Decision Trees, Random Forest, Gradient

normalizer = MinMaxScaler()

normalizer.fit(X_train)

X_train_norm = normalizer.transform(X_train) 

X_test_norm = normalizer.transform(X_test)

X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [25]:
# Apply Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=0)

random_forest.fit(X_train_norm, y_train)

y_pred = random_forest.predict(X_test_norm)


c:\Users\jesus\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


- Evaluate your model

In [26]:
# Evaluate the RandomForestClassifier model with thge accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.789712556732224


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [27]:
# Grid/Random Search, use GridSearchCV and define the hyperparameters to fine tune the model
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [1, 5]
}

grid_search = GridSearchCV(estimator=random_forest, param_grid=param_grid, cv=5)
grid_search.fit(X_train_norm, y_train)


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=0),
             param_grid={'max_depth': [1, 5], 'n_estimators': [50, 100]})

- Run Grid Search

In [28]:
# Run the model with the best hyperparameters
best_random_forest = grid_search.best_estimator_
y_pred = best_random_forest.predict(X_test_norm)


c:\Users\jesus\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


- Evaluate your model

In [29]:
# Evaluate the model with the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


Accuracy: 0.7715582450832073
